In [1]:
import wicked as w
import numpy as np
import re
from fractions import Fraction

w.reset_space()
w.add_space("c", "fermion", "occupied", list('ijklmn'))
w.add_space("a", "fermion", "general", list('uvwxyzrst'))
w.add_space("v", "fermion", "unoccupied", list('abcdef'))
wt = w.WickTheorem()

In [49]:
def split_single_tensor(tensor):
    """
    Split a single expression or string of the form
    >>> H^{a0,a3}_{a1,a2}
    into a tuple containing its label, the upper, and lower indices, i.e.,
    >>> ('H', [a0, a3], [a1, a2])
    If the label is 'F' or 'V', the upper and lower indices are swapped 
    (see 4c-dsrg-mrpt2.ipynb::dsrg_mrpt2_update to see why)
    """
    tensor = str(tensor)
    label, indices = str(tensor).split('^{')
    upper = indices.split('}_')[0].split(',')
    lower = indices.split('_{')[1].split('}')[0].split(',')
    if label in ['F', 'V']: 
        upper, lower = lower, upper
    return label, upper, lower

def get_unique_tensor_indices(tensor, unused_indices, index_dict):
    """
    >>> ('V', [a0, c0], [a1, a2]) -> 'uiwx'
    Get the indices of a tensor for use in an einsum contraction.
    For example, given the tensor from split_single_tensor, 
    the list of available indices and indices that have already been assigned, 
    we can generate the index string.
    """
    label, upper, lower = tensor
    indstr = ''
    for i in upper+lower:
        if index_dict.get(i):
            indstr += index_dict[i]
        else:
            index = unused_indices[i[0]].pop(0)
            index_dict[i] = index
            indstr += index
    
    return indstr

def get_tensor_slice(tensor, fmt):
    """
    >>> ('V', [a0, c0], [a1, a2]) -> "V['acaa']" (fmt = 'dict')
    >>> ('V', [a0, c0], [a1, a2]) -> "V[a,c,a,a]" (fmt = 'slice')
    """
    if tensor[0] in ['gamma1', 'eta1', 'lambda2', 'lambda3']:
        return tensor[0]
    else:
        if (fmt == 'dict'):
            return tensor[0] + "['" + ''.join([i[0] for i in tensor[1]]) + ''.join([i[0] for i in tensor[2]]) + "']"
        elif (fmt == 'slice'):
            if ('T' in tensor[0]): # T tensors are always particle-hole sized.
                return tensor[0] + '[' + ','.join(['h'+i[0] for i in tensor[1]]) + ',' + ','.join(['p'+i[0] for i in tensor[2]]) + ']'
            else:
                return tensor[0] + '[' + ','.join([i[0] for i in tensor[1]]) + ',' + ','.join([i[0] for i in tensor[2]]) + ']'

def get_lhs_tensor_name(tensor):
    """
    >>> ('V', [a0, c0], [a1, a2]) -> "Vacaa"
    """
    return tensor[0] + '_' + ''.join([i[0] for i in tensor[1]]) + ''.join([i[0] for i in tensor[2]])

def get_factor(expression):
    """
    Returns the prefactor of a right hand side expression, taking care of edge cases where an empty space (for +1.0) or a negative sign (for -1.0) are present.
    """
    factor = str(expression).split('+=')[-1].split(' ')[1]
    try:
        return float(Fraction(factor))
    except ValueError:
        if factor == '-':
            return -1.0
        else:
            return 1.0

def compile_einsum(expression, fmt='dict', tensor_name=None, antisym_in=''):
    """
    Compile an expression into a valid einsum expression.
    """
    unused_indices = {'a':list('uvwxyzrst'), 'c':list('ijklmn'), 'v':list('abcdef')}
    index_dict = {}

    antisym = ''

    lhs = expression.lhs()
    rhs = expression.rhs()

    factor = get_factor(expression)
    if (antisym_in == 'l' or antisym_in == 'r'):
        factor = -factor

    exstr = ''
    tenstr = ''

    for i in str(rhs).split(' '):
        _ = split_single_tensor(i)
        tenstr += get_tensor_slice(_, fmt) + ', '
        exstr += get_unique_tensor_indices(_, unused_indices, index_dict)
        exstr += ','

    exstr = exstr[:-1] + '->'
    tenstr += "optimize='optimal')"

    _ = split_single_tensor(lhs)
    if (_[1] != ['']):
        left = get_lhs_tensor_name(_)
        res_indx = get_unique_tensor_indices(_, unused_indices, index_dict)
        if (antisym_in == 'l'):
            res_indx = res_indx[1] + res_indx[0] + res_indx[2:]
        elif (antisym_in == 'r'):
            res_indx = res_indx[:2] + res_indx[3] + res_indx[2]
        elif (antisym_in == 'lr'):
            res_indx = res_indx[1] + res_indx[0] + res_indx[3] + res_indx[2]
        exstr += res_indx
        if (len(res_indx)==4 and antisym_in == ''): # the latter means this is the first pass
            block_indx = left.split('_')[1]
            if (block_indx[0] == block_indx[1]):
                antisym = 'l'
                if (block_indx[2] == block_indx[3]):
                    antisym += 'r'
            elif (block_indx[2] == block_indx[3]):
                antisym = 'r'
    else:
        left = _[0] # If it's scalar, just return the label.

    if tensor_name is not None:
        left = tensor_name

    einsumstr = left \
        + ' ' \
        + f"+= {factor:+.8f} * np.einsum('"\
        + exstr \
        + "', " \
        + tenstr

    return einsumstr, antisym    

def make_code(mbeq, fmt='dict', tensor_name=None):
    code = ''
    nlines = 0
    for i in mbeq:
        einsum, antisym = compile_einsum(i, fmt, tensor_name)
        code += '\t' + einsum + '\n'
        nlines += 1
        if (antisym != ''):
            if (False): # Turn it off for now.
                if (antisym == 'l'):
                    einsum, _ = compile_einsum(i, fmt, tensor_name, antisym_in='l')
                    code += '\t' + einsum + '\n'
                    nlines += 1
                elif (antisym == 'r'):
                    einsum, _ = compile_einsum(i, fmt, tensor_name, antisym_in='r')
                    code += '\t' + einsum + '\n'
                    nlines += 1
                elif (antisym == 'lr'):
                    einsum, _ = compile_einsum(i, fmt, tensor_name, antisym_in='l')
                    code += '\t' + einsum + '\n'
                    einsum, _ = compile_einsum(i, fmt, tensor_name, antisym_in='r')
                    code += '\t' + einsum + '\n'
                    einsum, _ = compile_einsum(i, fmt, tensor_name, antisym_in='lr')
                    code += '\t' + einsum + '\n'
                    nlines += 3

    return code, nlines

def get_elements(op1, op2, nbody):
    """
    Returns the elements of the commutator of two operators.
    """
    comm = w.commutator(op1, op2)
    comm_expr = wt.contract(comm, nbody*2, nbody*2)
    return comm_expr.to_manybody_equation("H")

def make_nbody_elements(op1, op2, nbody, fmt='slice'):
    """
    Returns the elements of the commutator of two operators in einsum format.
    """
    code = ''
    nlines = 0
    comm_expr = get_elements(op1, op2, nbody)

    for key in comm_expr.keys():
        if (nbody != 0):
            if (fmt == 'slice'):
                lhs_slice = re.findall(r'[a-zA-Z]', key)
                if (len(lhs_slice) == 4):
                    lhs_slice = ','.join(lhs_slice[:2]) + ',' + ','.join(lhs_slice[2:][::-1])
                else:
                    lhs_slice = ','.join(lhs_slice)
            _ = make_code(comm_expr[key], fmt, f'C{nbody}[{lhs_slice}]')
        else:
            _ = make_code(comm_expr[key], fmt, f'C{nbody}')
        code += _[0]
        nlines += _[1]
    return code, nlines

Here we first generate the MRPT2 energy and the AAAA block of $\bar{H}$

In [34]:
F = w.utils.gen_op('F',1,'cav','cav',diagonal=True)
V = w.utils.gen_op('V',2,'cav','cav',diagonal=True)
Hop = F + V

T1op = w.utils.gen_op('T1',1,'av','ca',diagonal=False)
T2op = w.utils.gen_op('T2',2,'av','ca',diagonal=False)
Top = T1op + T2op

HT = w.commutator(Hop, Top) # H1tilde has all blocks: note that the diagonal parts of Htilde1 are just the bare Hamiltonian
HTexpr = wt.contract(HT, 0, 4)
HTeq = HTexpr.to_manybody_equation("H")

In [39]:
HTeq.keys()

dict_keys(['aa|aa', 'aa|ac', 'aa|cc', 'aa|va', 'aa|vc', 'aa|vv', 'av|aa', 'av|ac', 'av|cc', 'av|va', 'av|vc', 'av|vv', 'a|a', 'a|c', 'a|v', 'ca|aa', 'ca|ac', 'ca|cc', 'ca|va', 'ca|vc', 'ca|vv', 'cc|aa', 'cc|ac', 'cc|cc', 'cc|va', 'cc|vc', 'cc|vv', 'cv|aa', 'cv|ac', 'cv|cc', 'cv|va', 'cv|vc', 'cv|vv', 'c|a', 'c|c', 'c|v', 'vv|aa', 'vv|ac', 'vv|va', 'vv|vc', 'vv|vv', 'v|a', 'v|c', 'v|v', '|'])

In [58]:
def make_contractions_file():
    input_dict = {
        'H_T_C0':   (Hop, Top, 0),
        'H1_T1_C1': (F, T1op, 1),
        'H1_T2_C1': (F, T2op, 1),
        'H2_T1_C1': (V, T1op, 1),
        'H2_T2_C1': (V, T2op, 1),
        'H1_T2_C2': (F, T2op, 2),
        'H2_T1_C2': (V, T1op, 2),
        'H2_T2_C2': (V, T2op, 2)
    }
    slicedef = '\thc = mf.hc\n\tha = mf.ha\n\tpa = mf.pa\n\tpv = mf.pv\n\tc = mf.core\n\ta = mf.active\n\tv = mf.virt\n'

    with open('wicked_contractions.py', 'w') as f:
        f.write('import numpy as np\nimport time\n\n')
        for key in input_dict.keys():
            code, nlines = make_nbody_elements(*input_dict[key], fmt='slice')
            f.write('def ' + key + '(C1, C2, F, V, T1, T2, gamma1, eta1, lambda2, lambda3, mf):\n')
            f.write('\t# ' + str(nlines) + ' lines\n')
            f.write('\tt0 = time.time()\n')
            if (key == 'H_T_C0'): f.write('\tC0 = .0j\n')
            f.write(slicedef+'\n')
            f.write(code + '\n')
            f.write('\tt1 = time.time()\n')
            f.write('\tprint("'+key+' took {:.4f} seconds to run.".format(t1-t0))\n\n')
            if (key == 'H_T_C0'): f.write('\treturn C0\n\n')

In [68]:
print(HTeq['ca|aa'][22])
print(compile_einsum(HTeq['ca|aa'][22])[0])

H^{c0,a0}_{a1,a2} += 1/4 T2^{c0,a0}_{a3,a4} V^{a5,a6}_{a1,a2} eta1^{a4}_{a6} eta1^{a3}_{a5}
H_caaa += +0.25000000 * np.einsum('iuvw,xyzr,wr,vz->iuxy', T2['caaa'], V['aaaa'], eta1, eta1, optimize='optimal')


In [69]:
type(HTeq['ca|aa'][22])

wicked._wicked.Equation

In [59]:
make_contractions_file()

Here we first show how $E^{(2)}$ can be generated.

In [6]:
print(make_code(HTeq['|'], fmt='slice')[0])

	H += +1.00000000 * np.einsum('iu,iv,vu->', F[c,a], T1[hc,pa], eta1, optimize='optimal')
	H += -0.50000000 * np.einsum('iu,ivwx,wxuv->', F[c,a], T2[hc,ha,pa,pa], lambda2, optimize='optimal')
	H += +1.00000000 * np.einsum('ia,ia->', F[c,v], T1[hc,pv], optimize='optimal')
	H += +1.00000000 * np.einsum('ua,va,uv->', F[a,v], T1[ha,pv], gamma1, optimize='optimal')
	H += -0.50000000 * np.einsum('ua,vwxa,uxvw->', F[a,v], T2[ha,ha,pa,pv], lambda2, optimize='optimal')
	H += -0.50000000 * np.einsum('iu,ivwx,uvwx->', T1[hc,pa], V[c,a,a,a], lambda2, optimize='optimal')
	H += -0.50000000 * np.einsum('ua,vwxa,vwux->', T1[ha,pv], V[a,a,a,v], lambda2, optimize='optimal')
	H += +0.25000000 * np.einsum('ijuv,ijwx,vx,uw->', T2[hc,hc,pa,pa], V[c,c,a,a], eta1, eta1, optimize='optimal')
	H += +0.12500000 * np.einsum('ijuv,ijwx,uvwx->', T2[hc,hc,pa,pa], V[c,c,a,a], lambda2, optimize='optimal')
	H += +0.50000000 * np.einsum('iuvw,ixyz,wz,vy,xu->', T2[hc,ha,pa,pa], V[c,a,a,a], eta1, eta1, gamma1, optimize='opt

Followed by the one- and two-body components of the all-active part of $\bar{H}^{(1)}$ required by reference relaxation

In [7]:
print(make_code(HTeq['a|a'], fmt='slice')[0])

	H_aa += -1.00000000 * np.einsum('iu,iv->uv', F[c,a], T1[hc,pa], optimize='optimal')
	H_aa += -1.00000000 * np.einsum('iu,ivwx,xu->vw', F[c,a], T2[hc,ha,pa,pa], eta1, optimize='optimal')
	H_aa += -1.00000000 * np.einsum('ia,iuva->uv', F[c,v], T2[hc,ha,pa,pv], optimize='optimal')
	H_aa += +1.00000000 * np.einsum('ua,va->vu', F[a,v], T1[ha,pv], optimize='optimal')
	H_aa += +1.00000000 * np.einsum('ua,vwxa,uw->vx', F[a,v], T2[ha,ha,pa,pv], gamma1, optimize='optimal')
	H_aa += -1.00000000 * np.einsum('iu,ivwx,ux->wv', T1[hc,pa], V[c,a,a,a], eta1, optimize='optimal')
	H_aa += -1.00000000 * np.einsum('ia,iuva->vu', T1[hc,pv], V[c,a,a,v], optimize='optimal')
	H_aa += +1.00000000 * np.einsum('ua,vwxa,wu->xv', T1[ha,pv], V[a,a,a,v], gamma1, optimize='optimal')
	H_aa += -0.50000000 * np.einsum('ijuv,ijwx,vx->wu', T2[hc,hc,pa,pa], V[c,c,a,a], eta1, optimize='optimal')
	H_aa += +0.50000000 * np.einsum('iuvw,ixyz,wxyz->uv', T2[hc,ha,pa,pa], V[c,a,a,a], lambda2, optimize='optimal')
	H_aa += -1.00000

In [8]:
print(make_code(HTeq['aa|aa'], fmt='slice',tensor_name='_V')[0])

	_V += -0.50000000 * np.einsum('iu,ivwx->uvwx', F[c,a], T2[hc,ha,pa,pa], optimize='optimal')
	_V += +0.50000000 * np.einsum('iu,ivwx->vuwx', F[c,a], T2[hc,ha,pa,pa], optimize='optimal')
	_V += +0.50000000 * np.einsum('iu,ivwx->uvxw', F[c,a], T2[hc,ha,pa,pa], optimize='optimal')
	_V += -0.50000000 * np.einsum('iu,ivwx->vuxw', F[c,a], T2[hc,ha,pa,pa], optimize='optimal')
	_V += -0.50000000 * np.einsum('ua,vwxa->vwux', F[a,v], T2[ha,ha,pa,pv], optimize='optimal')
	_V += +0.50000000 * np.einsum('ua,vwxa->wvux', F[a,v], T2[ha,ha,pa,pv], optimize='optimal')
	_V += +0.50000000 * np.einsum('ua,vwxa->vwxu', F[a,v], T2[ha,ha,pa,pv], optimize='optimal')
	_V += -0.50000000 * np.einsum('ua,vwxa->wvxu', F[a,v], T2[ha,ha,pa,pv], optimize='optimal')
	_V += -0.50000000 * np.einsum('iu,ivwx->wxuv', T1[hc,pa], V[c,a,a,a], optimize='optimal')
	_V += +0.50000000 * np.einsum('iu,ivwx->xwuv', T1[hc,pa], V[c,a,a,a], optimize='optimal')
	_V += +0.50000000 * np.einsum('iu,ivwx->wxvu', T1[hc,pa], V[c,a,a,a], opt

Now we generate the following elements
* $[\hat{H}_1,\hat{T}_1]_1$: H1_T1_C1
* $[\hat{H}_1,\hat{T}_2]_1$: H1_T2_C1
* $[\hat{H}_2,\hat{T}_1]_1$: H2_T1_C1
* $[\hat{H}_2,\hat{T}_2]_1$: H2_T2_C1
* $[\hat{H}_1,\hat{T}_2]_2$: H1_T2_C2
* $[\hat{H}_2,\hat{T}_1]_2$: H2_T1_C2
* $[\hat{H}_2,\hat{T}_2]_2$: H2_T2_C2

In [9]:
print(make_nbody_elements(F, T2op, 2))

("\tH2[a,a,a,a] += -0.50000000 * np.einsum('iu,ivwx->uvwx', F[c,a], T2[hc,ha,pa,pa], optimize='optimal')\n\tH2[a,a,a,a] += +0.50000000 * np.einsum('iu,ivwx->vuwx', F[c,a], T2[hc,ha,pa,pa], optimize='optimal')\n\tH2[a,a,a,a] += +0.50000000 * np.einsum('iu,ivwx->uvxw', F[c,a], T2[hc,ha,pa,pa], optimize='optimal')\n\tH2[a,a,a,a] += -0.50000000 * np.einsum('iu,ivwx->vuxw', F[c,a], T2[hc,ha,pa,pa], optimize='optimal')\n\tH2[a,a,a,a] += -0.50000000 * np.einsum('ua,vwxa->vwux', F[a,v], T2[ha,ha,pa,pv], optimize='optimal')\n\tH2[a,a,a,a] += +0.50000000 * np.einsum('ua,vwxa->wvux', F[a,v], T2[ha,ha,pa,pv], optimize='optimal')\n\tH2[a,a,a,a] += +0.50000000 * np.einsum('ua,vwxa->vwxu', F[a,v], T2[ha,ha,pa,pv], optimize='optimal')\n\tH2[a,a,a,a] += -0.50000000 * np.einsum('ua,vwxa->wvxu', F[a,v], T2[ha,ha,pa,pv], optimize='optimal')\n\tH2[a,a,a,c] += -0.50000000 * np.einsum('ia,uvwa->uviw', F[c,v], T2[ha,ha,pa,pv], optimize='optimal')\n\tH2[a,a,a,c] += +0.50000000 * np.einsum('ia,uvwa->vuiw', F[c,